# 02 - Convex Model: Portfolio Optimization

This notebook builds a convex mean-variance optimizer, tests convexity (Hessian), runs the optimizer for several risk-aversion values, and saves results to the `Results` folder.

In [ ]:
# Imports + load processed data
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cvxpy as cp

os.makedirs("../Results/plots", exist_ok=True)

mu = pd.read_json("../Results/processed/mean_returns.json", typ="series")
cov = pd.read_csv("../Results/processed/cov_matrix.csv", index_col=0)
tickers = pd.read_csv("../Results/processed/selected_tickers.csv", header=None)[0].tolist()


(A       0.000453
 AAL     0.001245
 AAP     0.000443
 AAPL    0.000786
 ABBV    0.001050
 dtype: float64,
              A       AAL       AAP      AAPL      ABBV       ABC       ABT  \
 Name                                                                         
 A     0.000240  0.000102  0.000055  0.000062  0.000092  0.000050  0.000088   
 AAL   0.000102  0.000504  0.000081  0.000069  0.000085  0.000072  0.000080   
 AAP   0.000055  0.000081  0.000359  0.000035  0.000067  0.000043  0.000065   
 AAPL  0.000062  0.000069  0.000035  0.000213  0.000045  0.000028  0.000048   
 ABBV  0.000092  0.000085  0.000067  0.000045  0.000284  0.000079  0.000090   
 
            ACN      ADBE       ADI  ...        XL      XLNX       XOM  \
 Name                                ...                                 
 A     0.000077  0.000091  0.000096  ...  0.000053  0.000087  0.000062   
 AAL   0.000076  0.000104  0.000105  ...  0.000080  0.000088  0.000043   
 AAP   0.000047  0.000050  0.000056  ...  

In [ ]:
# Convexity (Hessian PSD check)
Sigma = cov.loc[tickers, tickers].values
H = 2 * Sigma
eigenvalues = np.linalg.eigvalsh(H)  # symmetric eigvals
print("Eigenvalues (min/max):", float(eigenvalues.min()), float(eigenvalues.max()))
print("Is PSD? →", np.all(eigenvalues >= -1e-8))

# Small ridge for numerical stability
Sigma = Sigma + 1e-8 * np.eye(Sigma.shape[0])

Eigenvalues: [6.54703548e-02 1.22256585e-02 7.75461445e-03 4.95540415e-03
 4.09430506e-03 3.43207448e-03 2.91287722e-03 2.56856277e-03
 2.47472796e-03 2.28934949e-03 2.25842307e-03 2.10395645e-03
 1.93606045e-03 1.90568291e-03 1.82137695e-03 1.78853308e-03
 1.71612538e-03 1.60370822e-03 1.59563660e-03 1.51855671e-03
 1.50496386e-03 1.40289451e-03 1.38761825e-03 1.32688795e-03
 1.30861936e-03 1.26985465e-03 1.26070321e-03 1.19446336e-03
 1.17299057e-03 1.15502157e-03 1.12163188e-03 1.11011547e-03
 1.08856286e-03 1.04574158e-03 1.01104317e-03 1.00458020e-03
 9.92780665e-04 9.84431989e-04 9.52134200e-04 9.21395083e-04
 9.10874457e-04 8.91538298e-04 8.86589087e-04 8.83518129e-04
 8.63108818e-04 8.55667861e-04 8.34673068e-04 8.23968355e-04
 8.12237791e-04 8.06543765e-04 7.88378545e-04 7.69775020e-04
 7.51662712e-04 7.50357308e-04 7.35481109e-04 7.31567475e-04
 7.24845308e-04 7.12418677e-04 7.04650243e-04 6.97045469e-04
 6.86940030e-04 6.80153731e-04 6.67846737e-04 6.67964429e-04
 6.60161564

In [ ]:
# Solve convex mean-variance problem
n = len(tickers)
x = cp.Variable(n)
lam = 10
objective = cp.Minimize(cp.quad_form(x, Sigma) - lam * (mu.loc[tickers].values @ x))
constraints = [cp.sum(x) == 1, x >= 0, x <= 0.3]
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.OSQP, verbose=False)

weights = x.value
weights[:10]

Exception: Invalid dimensions for arguments to quad_form.

In [ ]:
# Allocation plot (saved)
plt.figure(figsize=(14,5))
sns.barplot(x=tickers, y=weights, color="steelblue")
plt.xticks(rotation=45, ha="right")
plt.title("Optimal Portfolio Weights (Convex Model)")
plt.tight_layout()
plt.savefig("../Results/plots/allocation_comparison.png", dpi=150)
plt.show()

In [ ]:
# Risk vs Return plot (saved)
expected_return = mu.loc[tickers].values @ weights
risk = float(weights.T @ Sigma @ weights)

plt.figure(figsize=(7,5))
plt.scatter(risk, expected_return, s=100, c="crimson")
plt.xlabel("Portfolio Risk")
plt.ylabel("Expected Return")
plt.title("Convex Model: Risk vs Return")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("../Results/plots/convex_risk_return.png", dpi=150)
plt.show()

print("Expected Return:", float(expected_return))
print("Risk:", risk)